In [ ]:
epochs = 50

# পর্ব 7 - ফেডারেটড্যাটাসেটের সাথে ফেডারেট লার্নিং

এখানে আমরা ফেডারেটেড ডেটাসেট ব্যবহারের জন্য একটি নতুন সরঞ্জাম প্রবর্তন করি। আমরা একটি "ফেডারেটড্যাটাসেট` ক্লাস তৈরি করেছি যা পাইটর্চ ডেটাসেট ক্লাসের মতো ব্যবহার করার উদ্দেশ্যে এবং এটি একটি ফেডারেশনযুক্ত ডেটা লোডার - ফেডারেটডাটাডোলোডারকে দেওয়া হয়েছে যা এটি ফেডারেশন ফ্যাশনে পুনরুত্থিত হবে।


লেখক:
- অ্যান্ড্রু ট্রস্ক - টুইটার: [@ আইয়ামট্রস্ক](https://twitter.com/iamtrask)
- থিও রাইফেল - গিটহাব: [@ ল্যারিফেল](https://github.com/LaRiffle)

অনুবাদক:
- সায়ন্তন দাস - িটহাব: [@ucalyptus](https://github.com/ucalyptus)

আমরা স্যান্ডবক্সটি ব্যবহার করি যা আমরা শেষ পাঠটি আবিষ্কার করেছি

In [ ]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

আমরা স্যান্ডবক্সটি ব্যবহার করি যা আমরা শেষ পাঠটি আবিষ্কার করেছি...

In [ ]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

আমরা একটি মডেল এবং একটি অপ্টিমাইজার লোড করি

In [ ]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

এখানে আমরা একটি `ফেডারেটড্যাটাসেটে প্রাপ্ত ডেটা কাস্ট করেছি` কর্মীদের দেখুন যা ডেটা অংশ রয়েছে।

In [ ]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

আমরা এটিকে একটি `ফেডারেটেড ডেটা লোডার এ রেখেছি এবং বিকল্পগুলি নির্দিষ্ট করি

In [ ]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

এবং অবশেষে আমরা যুগের উপর পুনরাবৃত্তি। খাঁটি এবং স্থানীয় পাইটর্চ প্রশিক্ষণের সাথে এটির তুলনা তুলনাযোগ্য দেখতে পাবেন!

In [ ]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)


# অভিনন্দন !!! - সম্প্রদায় যোগদানের সময়!

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন!

### গিটহাবে স্টার পাইসাইফ্ট

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হ'ল রেপোসকে অভিনীত করা! এটি আমরা যে শীতল সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [স্টার পাইসাইফ্ট](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাচে যোগ দিন!

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হ'ল আমাদের সম্প্রদায়ে যোগদান করা! আপনি [http://slack.openmined.org](http://slack.openmined.org) এ ফর্মটি পূরণ করে এটি করতে পারেন

### একটি কোড প্রকল্পে যোগদান করুন!

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হ'ল কোড অবদানকারী হয়ে উঠুন! যে কোনও সময় আপনি পাইসাইফ্ট গিটহাব ইস্যু পৃষ্ঠাতে যেতে পারেন এবং "প্রকল্পগুলি" এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষ স্তরের সমস্ত টিকিট দেখিয়ে দেবে যে আপনি কোন প্রকল্পগুলিতে যোগদান করতে পারেন তার একটি ওভারভিউ দেয়! আপনি যদি কোনও প্রকল্পে যোগ দিতে না চান তবে আপনি কিছুটা কোডিং করতে চান তবে আপনি আরও ভাল "ওয়ান অফ" মিনি-প্রকল্পগুলি "ভাল প্রথম ইস্যু" হিসাবে চিহ্নিত গিটহাব ইস্যুগুলি অনুসন্ধান করেও দেখতে পারেন।

- [পাইসাইফ্ট প্রজেক্টস](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [ভাল প্রথম ইস্যুর টিকিট](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন

আপনার যদি আমাদের কোডবেসে অবদান রাখার সময় না থাকে তবে তবুও সমর্থন leণ দিতে চান, আপনি আমাদের ওপেন কালেক্টিভেরও ব্যাকের হয়ে উঠতে পারেন। সমস্ত অনুদান আমাদের ওয়েব হোস্টিং এবং অন্যান্য সম্প্রদায় ব্যয় যেমন হ্যাকাথনস এবং মেটআপগুলির দিকে যায়!

[ওপেনমাইন্ডের মুক্ত সমাহারক পৃষ্ঠা](https://opencollective.com/openmined)